In [1]:
import torch
import torch.autograd as autograd # torch中自动计算梯度模块
import torch.nn as nn             # 神经网络模块
import torch.nn.functional as F   # 神经网络模块中的常用功能 
import torch.optim as optim       # 模型优化器模块
 
import time
T0 = time.time()
torch.manual_seed(1)

EMBEDDING_DIM = 6
HIDDEN_DIM =3

In [2]:
# class LSTMTagger(nn.Module):
 
#     def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
#         super(LSTMTagger, self).__init__()
#         self.hidden_dim = hidden_dim
 
#         self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
 
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim)
 
#         self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
#         self.hidden = self.init_hidden()
 
#     def init_hidden(self):
#         return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
#                 autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
 
#     def forward(self, sentence):
#         embeds = self.word_embeddings(sentence)
#         lstm_out, self.hidden = self.lstm(
#             embeds.view(len(sentence), 1, -1), self.hidden)
#         tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
#         tag_scores = F.log_softmax(tag_space)
#         return tag_scores

In [3]:
# lstm单元输入和输出维度都是3
lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM)
# 生成一个长度为5，每一个元素为1*3的序列作为输入，这里的数字3对应于上句中第一个3
inputs = [autograd.Variable(torch.randn((1, EMBEDDING_DIM)))for _ in range(3)]
    # 设置隐藏层维度，初始化隐藏层的数据
hidden = (autograd.Variable(torch.randn(1, 1, HIDDEN_DIM)),autograd.Variable(torch.randn((1, 1, HIDDEN_DIM))))
 
for i in inputs:
# Step through the sequence one element at a time.
# after each step, hidden contains the hidden state.
    print(f"-------------{i.view(1,1,-1).size()}")
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(f"out:{out.size()} hidden:{len(hidden)}")

-------------torch.Size([1, 1, 6])
out:torch.Size([1, 1, 3]) hidden:2
-------------torch.Size([1, 1, 6])
out:torch.Size([1, 1, 3]) hidden:2
-------------torch.Size([1, 1, 6])
out:torch.Size([1, 1, 3]) hidden:2


In [4]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_ix = {} # 单词的索引字典
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2} # 手工设定词性标签数据字典



class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
#         print("~~~~~~~", embeds.size(),type(embeds))
#         print("~~~~~~~", sentence, len(sentence)) #sentence,
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim = 1)
        return tag_scores


{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [5]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
print(time.time() - T0)

0.15135717391967773


In [6]:
T1 = time.time()

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
 
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(training_data[0][0])
print(inputs)
print(tag_scores.size())
print(time.time() - T1)

['The', 'dog', 'ate', 'the', 'apple']
Variable containing:
 0
 1
 2
 3
 4
[torch.LongTensor of size 5]

torch.Size([5, 3])
0.00824594497680664


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [7]:
T2 = time.time()
for epoch in range(300): # 我们要训练300次，可以根据任务量的大小酌情修改次数。
    for sentence, tags in training_data:
        # 清除网络先前的梯度值，梯度值是Pytorch的变量才有的数据，Pytorch张量没有
        model.zero_grad()
        # 重新初始化隐藏层数据，避免受之前运行代码的干扰
        model.hidden = model.init_hidden()
        # 准备网络可以接受的的输入数据和真实标签数据，这是一个监督式学习
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        # 运行我们的模型，直接将模型名作为方法名看待即可
        tag_scores = model(sentence_in)
        # 计算损失，反向传递梯度及更新模型参数
#         print("tag_scores",tag_scores.size(),type(tag_scores))
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

        # 来检验下模型训练的结果
        inputs = prepare_sequence(training_data[0][0], word_to_ix)
        tag_scores = model(inputs)
        if (0 == (epoch+1)% 100):
            print(tag_scores.size())
            print(epoch,"time elapsed",time.time() - T2)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([5, 3])
99 time elapsed 0.6308820247650146
torch.Size([5, 3])
99 time elapsed 0.6353158950805664
torch.Size([5, 3])
199 time elapsed 1.1080238819122314
torch.Size([5, 3])
199 time elapsed 1.1109139919281006
torch.Size([5, 3])
299 time elapsed 1.5669219493865967
torch.Size([5, 3])
299 time elapsed 1.5699918270111084
